In [1]:
import pandas as pd
import http.client
import json
import vnstock as vs

Get all company information

In [2]:
conn = http.client.HTTPSConnection("fiin-core.ssi.com.vn")
payload = ''
headers = {
  'authority': 'fiin-core.ssi.com.vn',
  'accept': 'application/json',
  'accept-language': 'en-US,en;q=0.9',
  'content-type': 'application/json',
  'dnt': '1',
  'origin': 'https://iboard.ssi.com.vn',
  'referer': 'https://iboard.ssi.com.vn/',
  'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-site',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0  uacq',
  'x-fiin-key': 'KEY',
  'x-fiin-seed': 'SEED',
  'x-fiin-user-id': 'ID'
}
conn.request("GET", "/Master/GetListOrganization?language=vi", payload, headers)
res = conn.getresponse()
data = res.read()
# 
# convert data into dataframe with the key 'items'
df_company_list = pd.DataFrame(json.loads(data)['items'])
df_company_list.set_index('ticker', inplace=True)

Set variable

In [6]:
#Declare variable to hold list of company 
ticker_list = list(df_company_list.reset_index()['ticker'])

#Set time for get stock data
start_period = pd.Period('2021-01', freq='Q-DEC')
end_period = pd.Period('2023-03', freq='Q-DEC')

In [158]:
#Get income statement
pd.set_option('display.max_columns', None)

df_concat = []

#Request API and append data
for i in ticker_list:
    try:
        df_data = vs.financial_flow(symbol=i, report_type='incomestatement', report_range='quarterly')[['ticker','postTaxProfit']]
        df_data.index = pd.to_datetime(df_data.index).to_period('Q')
        df_data = df_data[df_data.index >= start_period]
        df_data = df_data[df_data.index <= end_period]
        df_concat.append(df_data)
    except KeyError:
        pass
df_concat = pd.concat(df_concat)
df_incomestatement = df_concat.set_index(df_concat['ticker'], append=True)
del(df_incomestatement['ticker'])


In [159]:
#Create Year on Year and Quarter on Quarter column
df_incomestatement['Quarter On Quarter'] = df_incomestatement.groupby('ticker')['postTaxProfit'].shift(periods=-1)
df_incomestatement['Year on Year'] = df_incomestatement.groupby('ticker')['postTaxProfit'].shift(periods=-4)
df_incomestatement['YoY-ratio'] = (df_incomestatement['postTaxProfit']/df_incomestatement['Year on Year']) - 1

In [160]:
#Add year to date sum column
df_incomestatement.reset_index(inplace=True)
df_incomestatement['RecordYear'] = df_incomestatement['index'].dt.to_timestamp().dt.year
df_incomestatement.set_index(['RecordYear', 'ticker'], inplace=True)
df_incomestatement['YTD postTaxProfit'] = df_incomestatement.groupby(['RecordYear', 'ticker'])[['postTaxProfit']].sum()

In [161]:
test = ','.join(ticker_list)
i = 0

df_marketcap = []
while i <= len(ticker_list):
    try:
        df_data_mark = vs.stock_ls_analysis(','.join(ticker_list[i:i+2]))[['ticker','marcap', 'price']]
        df_marketcap.append(df_data_mark)
        i = i + 2
    except KeyError:
        i = i + 2
        pass

df_marketcap= pd.concat(df_marketcap)
df_marketcap.set_index('ticker', inplace=True)

In [162]:
df_financial = []



for i in ticker_list:
    try:
        df_data = vs.financial_ratio(i, 'quarterly', True)[['quarter','year','ticker', 'roe','priceToBook','earningPerShare', 'priceToEarning']]

        df_data['StockTime'] = df_data['year'].astype('str') +  'Q' + df_data['quarter'].astype('str')
        df_data.set_index('StockTime', inplace=True)
        df_data.index = pd.to_datetime(df_data.index).to_period('Q')
        del(df_data['quarter'])
        del(df_data['year'])
        df_data.set_index(df_data['ticker'], append=True, inplace=True)
        del(df_data['ticker'])

        df_data = df_data.astype({'earningPerShare':float, 'priceToEarning':float})

        df_data = df_data[df_data.index.get_level_values('StockTime') >= start_period]
        df_data = df_data[df_data.index.get_level_values('StockTime') <= end_period]

        df_financial.append(df_data)
    except KeyError:
        pass

df_financial = pd.concat(df_financial)

In [163]:
#Add sum of EPS across 4 latest quarter, EPS/4Q EPS
df_financial_final = df_financial.join(df_financial.reset_index('ticker').groupby('ticker')[['earningPerShare']].rolling(4).sum().shift(-3), rsuffix='-4Q')
df_financial_final['EPS/4Q EPS'] = df_financial_final['earningPerShare'] / df_financial_final['earningPerShare-4Q']

In [164]:
#Joining df_financial_final with df_incomestatement
df_info = df_incomestatement\
    .reset_index(level=0)\
    .rename(columns={'index':'StockTime'})\
    .set_index('StockTime', append=True)\
    .drop(columns='RecordYear')\
    .join(df_financial_final)

In [165]:
#Get a list of ticker base on marketcap
filter_marcap_list =  list(df_marketcap.query("marcap >= 1000").reset_index()['ticker'])

#Filter base on specific stocktime and  ticker_list and other conditions
filter_stock_time = pd.Period('2023-02', freq='Q-DEC')

df_info_filter = df_info[df_info.index.get_level_values('ticker').isin(filter_marcap_list)]\
    .query("`YoY-ratio` > 0.35 & earningPerShare > 800 & `EPS/4Q EPS` > 0.3")\
    .reset_index()\
    
df_final = df_info_filter[df_info_filter['StockTime'] == filter_stock_time]\
    .set_index('ticker')\
    .join(df_marketcap)\
    .join(df_company_list[['organShortName']])


In [166]:
df_final['Price / PriceToBook'] = df_final['price'] / df_final['priceToBook']
df_final['Predicted Price'] = (df_final['earningPerShare-4Q'] * df_final['Price / PriceToBook'] * 22.5)**(1/2)
df_final['Diff-ratio predicted vs current price'] = (df_final['Predicted Price']  / df_final['price']) - 1
df_final.sort_values('Diff-ratio predicted vs current price', ascending = False, inplace=True)

In [169]:
df_final.reset_index().to_excel('final.xlsx',index=False)